# Hito 3 - Clasificadores
## Para cada idioma, ¿somos capaces de ajustar un modelo predictivo que reciba un tweet y prediga su emoji asociado?

Las herramientas de procesamiento de texto natural han mostrado capacidades muy parecidas a las humanas. Testear su potencial en el contexto de este dataset es interesante puesto a que la variable a predecir es inherentemente subjetiva. En general, se espera que el emoji esté asociado al carácter emocional del tweet en cuestión, por ende tiene sentido testear modelos que han sido entrenados o ajustados para detectar sentimientos. No obstante, en este desafío hay emojis que presentan similar valor emocional. Además puede ser que el emoji corresponda a variables de mayor complejidad, como el sarcasmo del mensaje. Es por esto que el éxito en la predicción sería tarea difícil incluso para un humano.

Para responder a esta pregunta podemos usar modelos como Naïve Bayes, en el cual tomamos en consideración la ocurrencia de cada palabra en tweets de cada emoji, información que luego se usa para generar una probabilidad de emoji dado el tweet. También podría ser interesante usar modelos que tomen en consideración la interacción entre palabras. Un ejemplo de esto son los modelos de lenguaje. Podemos usar modelos de lenguajes pre-entrenados basados en redes neuronales, como es el caso de BERT/BETO, y ajustarlos para la predicción de emojis.


## Propuesta metodológica

Para responder a esta pregunta queremos usar distintos métodos de clasificación. Puede que algunos tengan más éxito que otros y es de nuestro interés analizar por qué, de ser el caso.

Como clasificadores hay muchos, usaremos los los de la lista siguiente:
- Naïve Bayes [(1)](#ref1)
- Clasificadores basados en Transformers

Antes que preferir una lista extensa de métodos, queremos analizar adecuadamente cada uno de ellos. Además, como este desafío se enmarcó en la competencia SEMEVAL, contamos con una extensa lista de competidores que incluye sus métricas globales de clasificación. Podemos, en la mayoría de los casos, averiguar qué método usaron. De este modo tendremos un análisis global del uso de diferentes métodos para clasificación multimodal de texto.

**Por qué y como usar Naïve Bayes?**

Creemos que este método, pese a su simpleza, puede dar resultados interesantes en esta tarea. Como ejemplo consideremos el siguiente tweet:

_Nearly halfway to **Christmas** 🎄 
Let me know, what's your favorite thing about **winter**?
Share this post with someone who celebrates **Christmas** all year round! 🎄_

Este tweet está relacionado con navidad, lo cual es evidente gracias a la presencia de ciertas palabras como: _christmas_ y _winter_. Como consecuencia, está muy propenso a que la clase en cuestión sea aquella del emoji _christmas_tree_, lo cual es efectivmente el caso. Si bien esto es menos claro para otros emoji, podemos generalizar esta idea y asumir que la clase del tweet estará dada por las palabras que lo componen. A su vez, cada palabra tendrá una probabilidad de pertenecer a las clases en cuestión.

El procedimiento para el clasificador será el siguiente:

``` 
Dado un parámetro alfa y un vocabulario, ajustamos un clasificador Naïve Bayes en base al conjunto de entrenamiento. Luego testeados la calidad de su evaluación con diferentes métricas usando el conjuntos de prueba (test).
```

Una razón para el uso de este método es que los resultados de Naïve Bayes son altamente interpretables puesto a que a cada palabra se le asigna la probabilidad de pertenecer a las distintas clases. Esto nos da un eje extra de exploración que usaremos del siguiente modo, dado un clasificador entrenado : 

```
Para cada emoji, seleccionar k palabras con probabilidad más alta de ser catalogadas con el emoji.
```

Otra manera de interpretar los resultados del clasificador es el siguiente: para cada palabra poseemos la probabilidad de pertenecer a alguna clase (emoji). Como tenemos 20 emojis (19 respectivamente), entonces esto nos dota de un vector 20-dimensional (resp. 19-d) a valores en [0,1]. Esto nos permite usar alguna técnica de reducción de dimensionalidad para visualizar el espacio que se genera con tales representaciones. Para esta tarea elegimos umap, pues algunos de los miembros del grupo ya tienen vasta experiencia usándola y afinando sus hiperparámetros. En resumen realizamos lo siguiente:

```
Para cada palabra del vocabulario, tomar su vector n-dimensional dado por la probabilidad de pertenecer a las n clases. Realizar una reducción de dimensión usando UMAP y proyectalos en el plano junto con el color de la clase más probable y con tamaño del punto dependiendo de cuan fuerte es la probabilidad de pertenecer a su clase más probable. Realizar un análisis cualitativo.
```

Para la implementación del clasificador usaremos la librería scikitlearn. Un parámetro a ajustar es el alpha. Este corresponde a la suavización de la verosimilitud, que está dada por la ecuación siguiente:

$$ \theta_{y^i} = \frac{N_{y^i}+\alpha}{N_y+\alpha n} $$

Donde $N_{y^i}$ es el número de veces que la palabra $i$ aparece en la clase $y$, y $N_y$ es el conteo total de palabras para la clase $y$. El valor $\theta_{y^i}$ corresponde a la probabilidad de que una palabra $i$ aparezca en la clase (emoji) $y$.

Por otro lado, la definición del vocabulario es importante a la hora de usar NB. Usamos también la librería scikitlearn para esto. Esta posee un parámetro min_df, que corresponde a la mínima cantidad de ocurrencia que debe tener una palabra para que está sea tomada en cuenta en el clasificador. De este modo, un min_df = 1 tomara todas las palabras. Usar un min_df más elevado nos permitirá analizar solo aquellas palabras que suceden seguido y por ende que portarán más información acerca de la pertenencia o no a una cierta clase.

Realizaremos un grid search para ajustar ambos parámetros en nuestro clasificador. Exploraremos los valores siguientes:

$$\alpha \in \[0.0,0.2,0.4,0.6,0.8,1.0\]$$

$$min\textunderscore df \in \[ 1,2,3,4,5,6,7,8,9,10 \] $$

Y escogeremos el ganador en base a la métrica macro f1 para ser consistentes con el resultado de la competición SEMEVAL.

```
Para distintos valores de alpha y min_df, entrenar un clasificador NB y escoger aquel con mayor macro f1
```

**Por qué y como usar Transformers?**

Transformers [2] es una arquitectura de redes neuronales que ha mostrado una enorme capacidad de modelar texto. Su uso se ha masificado y simplificado gracias a la existencia de bibliotecas como _transformers_, que ponen a libre disposición modelos pre-entrenados. Este punto es importante puesto a que entrenar un modelo de lenguaje robusto desde cero puede tomar tiempo y capacidad de cómputo que van más allá de nuestras capacidades.

Creemos que explorar su uso y compararlo con un clasificador como Naïve Bayes será una buena manera de complementar los conocimientos adquiridos en el curso con recursos más avanzados de interés personal. Sin embargo nos centraremos en modelos pre-entrenados sin ajustar. En particular analizaremos los siguientes:
- [BERTweet base (vinai)](https://huggingface.co/docs/transformers/model_doc/bertweet) [(3)](#ref3)
- [BERTweet ajustado para predicción de emojis (CardiffNLP)](https://huggingface.co/cardiffnlp/bertweet-base-emoji)
- [Twitter-roberta base (CardiffNLP)](https://huggingface.co/cardiffnlp/twitter-roberta-base) [(4)](#ref4)
- [Twitter-roberta ajustado para predicción de emojis (CardiffNLP)](https://huggingface.co/cardiffnlp/twitter-roberta-base-emoji)
- [BETO base (DCC-UChile)](https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased) [(5)](#ref5)

Esta eleccion nos permitirá comparar el efecto de hacer un ajuste a la tarea de predicción de emojis para dos modelos de transformers distintos en inglés. Por otro lado, testearemos el uso de un modelo de lenguaje sin ajustar para los tweets en español. Se hará un análisis de las predicciones consistente con el que usamos en Naïve Bayes, haciendo uso de las métricas de clasificación que provee la biblioteca _scikitlearn_.

Por otro lado, se buscarán modos de interpretación al análizar algunas capas de atención de los modelos. Para esto usaremos la herramienta open source [_bertviz_](https://github.com/jessevig/bertviz) [(6)](#ref6) que nos provee de una herramienta de visualización interactiva.

Finalmente, cabe señalar que hay una conexión directa con la pregunta 2, que pretende analizar clusters usando distintas representaciones de los tweets. En efecto, los modelos de transformers anteriormente mencionados nos otorgan un vector por tweet que puede será usado con este efecto. De este modo estaremos explorando también el clasificador en cuestión a través del espacio semántico que define.

**Referencias**

<a name="ref1"></a>[1] Metsis, V., Androutsopoulos, I., & Paliouras, G. (2006, July). Spam filtering with naive bayes-which naive bayes?. In CEAS (Vol. 17, pp. 28-69).

<a name="ref2"></a>[2] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., Kaiser, Ł., & Polosukhin, I. (2017). Attention is All you Need. Advances in Neural Information Processing Systems, 30, 5998–6008. https://arxiv.org/abs/1706.03762

<a name="ref3"></a>[3] Nguyen, D. Q., Vu, T., & Nguyen, A. T. (2020, October). BERTweet: A pre-trained language model for English Tweets. In Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing: System Demonstrations (pp. 9-14). https://aclanthology.org/2020.emnlp-demos.2.pdf

<a name="ref4"></a>[4] Barbieri, F., Camacho-Collados, J., Anke, L. E., & Neves, L. (2020, November). TweetEval: Unified Benchmark and Comparative Evaluation for Tweet Classification. In Findings of the Association for Computational Linguistics: EMNLP 2020 (pp. 1644-1650). https://arxiv.org/pdf/2010.12421.pdf

<a name="ref5"></a>[5] Canete, J., Chaperon, G., Fuentes, R., Ho, J. H., Kang, H., & Pérez, J. (2020). Spanish pre-trained bert model and evaluation data. Pml4dc at iclr, 2020, 2020. https://users.dcc.uchile.cl/~jperez/papers/pml4dc2020.pdf

<a name="ref6"></a>[6] Vig, J. (2019, July). A Multiscale Visualization of Attention in the Transformer Model. In Proceedings of the 57th Annual Meeting of the Association for Computational Linguistics: System Demonstrations (pp. 37-42). https://aclanthology.org/P19-3007.pdf


## Inglés

Realizaremos un resumen de las métricas usando los distintos clasificadores propuestos.

### Naive Bayes

Este modelo está basado en el resultado del grid-search realizado en la sección corresondiente.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

from config import *

df_us_train = pickle.load(open(file_names["df_us_train"], "rb"))
df_us_test = pickle.load(open(file_names["df_us_test"], "rb"))

In [2]:
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()

df_us_train['tokenized_text'] = df_us_train['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))
df_us_test['tokenized_text'] = df_us_test['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))

In [3]:
best_min_df = 10
best_alpha = 0.2

vectorizer = CountVectorizer(min_df=best_min_df)
X_train_bow = vectorizer.fit_transform(df_us_train["tokenized_text"])
X_test_bow = vectorizer.transform(df_us_test["tokenized_text"])

clf = MultinomialNB(alpha=best_alpha)
clf.fit(X_train_bow, df_us_train["label"])

y_pred_NB = clf.predict(X_test_bow)
# print("Métricas de clasificación usando Naive-Bayes")
# print(classification_report(df_us_test["label"], y_pred_NB, target_names=df_us_mapping["emoji"]))
dict_NB = classification_report(df_us_test["label"], y_pred_NB, target_names=df_us_mapping["emoji"],output_dict=True)

### Transformers

In [4]:
from os import listdir

modelos = [f.replace('.pickle','') for f in listdir('resultados_test/')]
print("Modelos disponibles:")
for m in modelos: print("\t- "+m)

Modelos disponibles:
	- beto-emoji
	- twitter-roberta-base
	- bertweet-base


In [5]:
preds_transformer = {}
modelos.remove('beto-emoji')
for m in modelos:
    with open('resultados_test/{}.pickle'.format(m), 'rb') as handle:
        list_pred = pickle.load(handle)
        preds_transformer[m] = [str(i) for i in list_pred]

In [6]:
dict_results = {}
for m in modelos:
    # print("Métricas de clasificación usando Transformers-{}".format(m))
    # print(classification_report(df_us_test["label"], preds_transformer[m], target_names=df_us_mapping["emoji"]))
    dict_results[m] = classification_report(df_us_test["label"], preds_transformer[m], target_names=df_us_mapping["emoji"],output_dict=True)

dict_results['Naive-Bayes'] = dict_NB

## Resultados

In [7]:
import pandas as pd

clfs = modelos + ['Naive-Bayes']
cols = pd.MultiIndex.from_product([clfs, ['precission', 'recall', 'f1-score']])

df_en = pd.DataFrame(columns=cols)

In [8]:
for lab in df_us_mapping["emoji"].values:
  values = []
  for m in clfs:
    values += list(dict_results[m][lab].values())[:3]
  df_en.loc[lab] = values

df_en

twitter-roberta-base                     bertweet-base                      \
             precission    recall  f1-score    precission    recall  f1-score   
❤              0.741124  0.862197  0.797089      0.812185  0.851824  0.831533   
😍              0.320503  0.443064  0.371948      0.327511  0.527950  0.404249   
📷              0.303647  0.697626  0.423126      0.308837  0.710196  0.430476   
🇺🇸             0.722648  0.532068  0.612884      0.815058  0.572088  0.672294   
☀              0.713768  0.622925  0.665260      0.745455  0.583399  0.654545   
💜              0.344262  0.018851  0.035745      0.230769  0.013465  0.025445   
😉              0.161893  0.099541  0.123281      0.177606  0.070444  0.100877   
💯              0.337004  0.122990  0.180212      0.294221  0.270096  0.281643   
😁              0.143443  0.030356  0.050107      0.137681  0.049436  0.072750   
🎄              0.638710  0.768932  0.697797      0.650188  0.783172  0.710511   
📸              0.473282  0.025652  0.048666      0.394464  0.047166  0.084257   
😜              0.101449  0.013861  0.024390      0.250000  0.000990  0.001972   
😂              0.452065  0.533524  0.489428      0.419958  0.663652  0.514403   
💕              0.275835  0.136276  0.182425      0.280054  0.238772  0.257770   
🔥              0.577502  0.526372  0.550753      0.743255  0.444833  0.556566   
😊              0.135502  0.265964  0.179535      0.157165  0.189709  0.171910   
😎              0.185644  0.225451  0.203620      0.214162  0.246994  0.229409   
✨              0.306567  0.419425  0.354224      0.342202  0.407057  0.371823   
💙              0.239531  0.092318  0.133271      0.340824  0.058748  0.100220   
😘              0.181388  0.195745  0.188293      0.180762  0.238298  0.205580   

   Naive-Bayes                      
    precission    recall  f1-score  
❤     0.376373  0.482404  0.422843  
😍     0.260080  0.243064  0.251284  
📷     0.143106  0.171788  0.156141  
🇺🇸    0.420067  0.517701  0.463801  
☀     0.228070  0.493281  0.311922  
💜     0.228758  0.062837  0.098592  
😉     0.101747  0.075804  0.086880  
💯     0.208511  0.196945  0.202563  
😁     0.103937  0.057242  0.073826  
🎄     0.566016  0.638188  0.599939  
📸     0.263504  0.149359  0.190652  
😜     0.065817  0.030693  0.041864  
😂     0.328016  0.467137  0.385406  
💕     0.181971  0.082917  0.113924  
🔥     0.453903  0.449139  0.451508  
😊     0.087386  0.071296  0.078525  
😎     0.148607  0.120240  0.132927  
✨     0.267382  0.208439  0.234260  
💙     0.189633  0.096837  0.128205  
😘     0.128894  0.102128  0.113960

Resumen de resultados globales

In [9]:
df_en_summary = pd.DataFrame(columns=['accuracy','macro f1','weighted f1','macro precision','weighted precision','macro recall','weighted recall'])

for m in clfs:
    values = []
    values += [dict_results[m]['accuracy']]
    values += [dict_results[m]['macro avg']['f1-score']] + [dict_results[m]['weighted avg']['f1-score']]
    values += [dict_results[m]['macro avg']['precision']] + [dict_results[m]['weighted avg']['precision']]
    values += [dict_results[m]['macro avg']['recall']] + [dict_results[m]['weighted avg']['recall']]
    df_en_summary.loc[m] = values

df_en_summary

,accuracy,macro f1,weighted f1,macro precision,weighted precision,macro recall,weighted recall
twitter-roberta-base,0.46024,0.315603,0.431739,0.367788,0.452797,0.331657,0.46024
bertweet-base,0.48032,0.333912,0.456200,0.391118,0.486150,0.348414,0.48032
Naive-Bayes,0.30642,0.226951,0.287651,0.237589,0.284859,0.235872,0.30642


## Español

In [10]:
df_es_train = pickle.load(open(file_names["df_es_train"], "rb"))
df_es_test = pickle.load(open(file_names["df_es_test"], "rb"))

df_es_train['tokenized_text'] = df_es_train['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))
df_es_test['tokenized_text'] = df_es_test['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))

In [11]:
best_min_df = 5
best_alpha = 0.2

vectorizer = CountVectorizer(min_df=best_min_df)
X_train_bow_es = vectorizer.fit_transform(df_es_train["tokenized_text"])
X_test_bow_es = vectorizer.transform(df_es_test["tokenized_text"])

clf_es = MultinomialNB(alpha=best_alpha)
clf_es.fit(X_train_bow_es, df_es_train["label"])

y_pred_NB_es = clf_es.predict(X_test_bow_es)

dict_NB_es = classification_report(df_es_test["label"], y_pred_NB_es, target_names=df_es_mapping["emoji"],output_dict=True)

In [12]:
with open('resultados_test/beto-emoji.pickle', 'rb') as handle:
    list_pred = pickle.load(handle)
    preds_beto = [str(i) for i in list_pred]

dict_results = {}
dict_results['beto-emoji'] = classification_report(df_es_test["label"], preds_beto, target_names=df_es_mapping["emoji"],output_dict=True)

dict_results['Naive-Bayes'] = dict_NB_es

/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [14]:
clfs_es = ['beto-emoji','Naive-Bayes']
cols = pd.MultiIndex.from_product([clfs_es, ['precission', 'recall', 'f1-score']])

df_es = pd.DataFrame(columns=cols)
for lab in df_es_mapping["emoji"].values:
  values = []
  for m in clfs_es:
    values += list(dict_results[m][lab].values())[:3]
  df_es.loc[lab] = values

df_es

beto-emoji                     Naive-Bayes                    
   precission    recall  f1-score  precission    recall  f1-score
❤    0.387432  0.434844  0.409771    0.355032  0.387202  0.370420
😍    0.289291  0.385653  0.330594    0.281461  0.295455  0.288288
😎    0.117647  0.106195  0.111628    0.130742  0.109145  0.118971
💙    0.360000  0.021792  0.041096    0.126984  0.038741  0.059369
💜    0.000000  0.000000  0.000000    0.100000  0.029787  0.045902
😜    0.035461  0.018248  0.024096    0.114286  0.058394  0.077295
💞    0.000000  0.000000  0.000000    0.000000  0.000000  0.000000
✨    0.260204  0.122596  0.166667    0.218447  0.108173  0.144695
🎶    0.248756  0.235849  0.242131    0.133987  0.193396  0.158301
💘    0.000000  0.000000  0.000000    0.050847  0.022388  0.031088
😁    0.049296  0.033493  0.039886    0.050633  0.038278  0.043597
😂    0.509030  0.507672  0.508350    0.437433  0.545697  0.485604
💕    0.087432  0.045455  0.059813    0.046823  0.039773  0.043011
😊    0.117126  0.231518  0.155556    0.106129  0.138132  0.120034
😘    0.236041  0.234257  0.235145    0.197568  0.163728  0.179063
💪    0.366667  0.429967  0.395802    0.284987  0.364821  0.320000
😉    0.147002  0.167770  0.156701    0.108040  0.094923  0.101058
👌    0.088608  0.155556  0.112903    0.084746  0.111111  0.096154
🇪🇸   0.462830  0.455189  0.458977    0.293919  0.410377  0.342520

In [15]:
df_es_summary = pd.DataFrame(columns=['accuracy','macro f1','weighted f1','macro precision','weighted precision','macro recall','weighted recall'])

for m in clfs_es:
    values = []
    values += [dict_results[m]['accuracy']]
    values += [dict_results[m]['macro avg']['f1-score']] + [dict_results[m]['weighted avg']['f1-score']]
    values += [dict_results[m]['macro avg']['precision']] + [dict_results[m]['weighted avg']['precision']]
    values += [dict_results[m]['macro avg']['recall']] + [dict_results[m]['weighted avg']['recall']]
    df_es_summary.loc[m] = values

df_es_summary

,accuracy,macro f1,weighted f1,macro precision,weighted precision,macro recall,weighted recall
beto-emoji,0.3050,0.181532,0.289702,0.198043,0.294545,0.188740,0.3050
Naive-Bayes,0.2735,0.159230,0.258523,0.164319,0.252616,0.165764,0.2735
